### Recursion

How do you implement recursion in a Turing machine? Here's how not to do it: We've occasionally made use of a convention where a TM $M$ has a step that says "Simulate $M'$." This is a macro rather than a function: you would code this in the formal description by including the definition of $M'$ in the definition of $M$. So if $M$ tried to invoke itself, you would end up with an infinite formal description.

We can do this in two ways. One way would be to implement a general function calling mechanism, with a call stack and so on, and then it would be no problem for a function to call itself.

A cooler way is described in Section 6.1. Suppose that we adopted the following convention: Whenever we invoke $M'$ on input $w$, we always additionally provide it with the encoding of itself. That is, before invoking $M'$ on $w$, we replace $w$ with $\langle M', w \rangle$. Then, when $M'$ wants to invoke itself, it can use a universal TM and $\langle M' \rangle$ to do it. Infinity avoided.

In Python, this would look like:

In [1]:
def factorial1(recurse, n):
    if n <= 1:
        return 1
    else:
        return n * recurse(recurse, n-1)
    
factorial1(factorial1, 5)

120

However, it's a nuisance for anyone who invokes $M'$ to bear the responsibility of providing it with $\langle M' \rangle$. So we can wrap $M'$ so that this happens automatically. As a first try, we could do:

In [2]:
def b(m):
    def r(w):
        return m(m, w)
    return r

factorial2 = b(factorial1)

factorial2(5)

120

But this still requires $M'$ to provide its own recursive calls with an encoding of itself. We can fix that, too:

In [4]:
def fixpoint(t):
    """Given a function t that takes two arguments, return a function r such that r(w) = t(r, w)."""
    def bt(m, w):
        r = b(m)
        return t(r, w)
    def r(w):
        return bt(bt, w)
    return r

@fixpoint
def factorial3(recurse, n):
    if n <= 1:
        return 1
    else:
        return n * recurse(n-1)
    
factorial3(5)

120

This is closely related to the [Y combinator](https://en.wikipedia.org/wiki/Fixed-point_combinator) after which the famous startup incubator was named. The construction in the book does this with Turing machines. Here is the book's construction, presented (I think) in a cleaner way:

Define a TM $B$ that, on input $\langle M, w \rangle$,
1. Constructs a TM $R$ that, on input $w$,
  1. Rewrites the tape to $\langle M, w \rangle$.
  2. Runs $M$ on the resulting tape.
2. Writes $\langle R, w\rangle$.

If $T$ is a TM that takes input $\langle R, w\rangle$, then let $BT$ be the TM that runs $B$ followed by $T$. Construct a TM $R$ that, on input $w$:
1. Rewrites the tape to $\langle BT, w \rangle$.
2. Runs $BT$ on the resulting tape.